## Author: Blaine Hill

In this notebook, we program out how to embed a KG such as FB15k_237 using RotatE. The weights are saved under embedding_model_weights.pth

In [21]:
import os
os.environ["WANDB_NOTEBOOK_NAME"] = "embedding_model.ipynb"

import wandb
wandb.login()

True

In [22]:
import torch
from torch_geometric.datasets import FB15k_237
from torch_geometric.nn import RotatE
from torch_geometric.loader import DataLoader
import torch.optim as optim

import os
import os.path as osp

In [23]:

from datetime import datetime
from torch_geometric.datasets import FB15k_237
from torch_geometric.nn import ComplEx

dataset_name='FB15k_237'
embedding_model_name='RotatE'

device = 'cuda' if torch.cuda.is_available() else 'cpu'
path = osp.join(os.getcwd(), '..', 'data', dataset_name)

train_data = FB15k_237(path, split='train')[0].to(device)
val_data = FB15k_237(path, split='val')[0].to(device)
test_data = FB15k_237(path, split='test')[0].to(device)

In [24]:
wandb.init(
    project=f"ScoreMatchingDiffKG_Embedding",
    name=f"{dataset_name}_{embedding_model_name}_embedding_model {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
    config={
        "epochs": 500,
        "batch_size": 1000,
        "lr": 0.001,
        "weight_decay": 1e-6,
        "k": 10 #used for top-k evaluation
    }
)

config = wandb.config

In [25]:
model = RotatE(num_nodes=train_data.num_nodes, num_relations=train_data.num_edge_types, hidden_channels=50).to(device) 

wandb.watch(model) #tracks gradients

[]

In [26]:
# Train model on FB15k dataset
train_loader = model.loader(
    head_index=train_data.edge_index[0],
    rel_type=train_data.edge_type,
    tail_index=train_data.edge_index[1],
    batch_size=config.batch_size,
    shuffle=True,
)
# val_loader = model.loader(
#     head_index=val_data.edge_index[0],
#     rel_type=val_data.edge_type,
#     tail_index=val_data.edge_index[1],
#     batch_size=config.batch_size,
#     shuffle=True,
# )
# test_loader = model.loader(
#     head_index=test_data.edge_index[0],
#     rel_type=test_data.edge_type,
#     tail_index=test_data.edge_index[1],
#     batch_size=config.batch_size,
#     shuffle=True,
# )
optimizer = optim.Adagrad(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)

In [27]:
def train(dataloader):
    model.train()
    total_loss = total_examples = 0
    for head_index, rel_type, tail_index in dataloader:
        optimizer.zero_grad()
        loss = model.loss(head_index, rel_type, tail_index)
        loss.backward()
        optimizer.step()
        #scale on specific number of elements in batch
        total_loss += float(loss) * head_index.numel()
        total_examples += head_index.numel()
    return total_loss / total_examples

In [28]:
@torch.no_grad()
def test(data):
    model.eval()
    # Calculate loss
    loss = model.loss(
        head_index=data.edge_index[0],
        rel_type=data.edge_type,
        tail_index=data.edge_index[1],
    )
    # Evaluate the model using model.test()
    mean_rank, mrr, hits_at_k = model.test(
        head_index=data.edge_index[0],
        rel_type=data.edge_type,
        tail_index=data.edge_index[1],
        batch_size=config.batch_size,
        k=config.k,
    )
    return loss, mean_rank, mrr, hits_at_k

In [29]:
for epoch in range(config.epochs):
    epoch += 1 #for string printing
    loss = train(train_loader)
    print(f'Epoch: {epoch:03d}, Train Loss: {loss:.4f}')
    train_metrics ={
        "train_epoch": epoch-1, #fixes the index
        "train_loss": loss
    }
    if epoch % 25 == 0:
        loss, mean_rank, mrr, hits_at_10 = test(val_data)
        print(f'Val Mean Rank: {mean_rank:.2f}, Val Mean Reciprocal Rank: {mrr:.2f}, Val Hits@10: {hits_at_10:.4f}')
        val_metrics ={
            "val_loss": loss,
            "val_mean_rank": mean_rank,
            "val_mrr": mrr,
            "val_hits_at_10": hits_at_10
        }
    #log to wandb
    wandb.log({**train_metrics, **val_metrics} if 'val_metrics' in locals() else {**train_metrics})

#once everything is finished, test model
loss, mean_rank, mrr, hits_at_10 = test(test_data)
print(f'Test Mean Rank: {mean_rank:.2f}, Test Mean Reciprocal Rank: {mrr:.2f}, Test Hits@10: {hits_at_10:.4f}')
test_metrics ={
            "test_loss": loss,
            "test_mean_rank": mean_rank,
            "test_mrr": mrr,
            "test_hits_at_10": hits_at_10
        }
wandb.log({**test_metrics})

Epoch: 001, Train Loss: 0.7774
Epoch: 002, Train Loss: 0.7709
Epoch: 003, Train Loss: 0.7668
Epoch: 004, Train Loss: 0.7635
Epoch: 005, Train Loss: 0.7608
Epoch: 006, Train Loss: 0.7584
Epoch: 007, Train Loss: 0.7562
Epoch: 008, Train Loss: 0.7542
Epoch: 009, Train Loss: 0.7524
Epoch: 010, Train Loss: 0.7507
Epoch: 011, Train Loss: 0.7491
Epoch: 012, Train Loss: 0.7477
Epoch: 013, Train Loss: 0.7462
Epoch: 014, Train Loss: 0.7449
Epoch: 015, Train Loss: 0.7436
Epoch: 016, Train Loss: 0.7423
Epoch: 017, Train Loss: 0.7411
Epoch: 018, Train Loss: 0.7400
Epoch: 019, Train Loss: 0.7388
Epoch: 020, Train Loss: 0.7378
Epoch: 021, Train Loss: 0.7367
Epoch: 022, Train Loss: 0.7357
Epoch: 023, Train Loss: 0.7348
Epoch: 024, Train Loss: 0.7338
Epoch: 025, Train Loss: 0.7329


100%|██████████| 17535/17535 [04:08<00:00, 70.62it/s]


Val Mean Rank: 4517.82, Val Mean Reciprocal Rank: 0.00, Val Hits@10: 0.0018
Epoch: 026, Train Loss: 0.7320
Epoch: 027, Train Loss: 0.7310
Epoch: 028, Train Loss: 0.7302
Epoch: 029, Train Loss: 0.7294
Epoch: 030, Train Loss: 0.7286
Epoch: 031, Train Loss: 0.7278
Epoch: 032, Train Loss: 0.7269
Epoch: 033, Train Loss: 0.7262
Epoch: 034, Train Loss: 0.7254
Epoch: 035, Train Loss: 0.7247
Epoch: 036, Train Loss: 0.7240
Epoch: 037, Train Loss: 0.7233
Epoch: 038, Train Loss: 0.7225
Epoch: 039, Train Loss: 0.7219
Epoch: 040, Train Loss: 0.7212
Epoch: 041, Train Loss: 0.7205
Epoch: 042, Train Loss: 0.7198
Epoch: 043, Train Loss: 0.7192
Epoch: 044, Train Loss: 0.7186
Epoch: 045, Train Loss: 0.7179
Epoch: 046, Train Loss: 0.7173
Epoch: 047, Train Loss: 0.7166
Epoch: 048, Train Loss: 0.7161
Epoch: 049, Train Loss: 0.7155
Epoch: 050, Train Loss: 0.7149


100%|██████████| 17535/17535 [04:07<00:00, 70.91it/s]


Val Mean Rank: 3340.89, Val Mean Reciprocal Rank: 0.02, Val Hits@10: 0.0376
Epoch: 051, Train Loss: 0.7143
Epoch: 052, Train Loss: 0.7137
Epoch: 053, Train Loss: 0.7132
Epoch: 054, Train Loss: 0.7126
Epoch: 055, Train Loss: 0.7121
Epoch: 056, Train Loss: 0.7116
Epoch: 057, Train Loss: 0.7110
Epoch: 058, Train Loss: 0.7104
Epoch: 059, Train Loss: 0.7100
Epoch: 060, Train Loss: 0.7094
Epoch: 061, Train Loss: 0.7089
Epoch: 062, Train Loss: 0.7085
Epoch: 063, Train Loss: 0.7080
Epoch: 064, Train Loss: 0.7074
Epoch: 065, Train Loss: 0.7069
Epoch: 066, Train Loss: 0.7065
Epoch: 067, Train Loss: 0.7059
Epoch: 068, Train Loss: 0.7055
Epoch: 069, Train Loss: 0.7050
Epoch: 070, Train Loss: 0.7046
Epoch: 071, Train Loss: 0.7041
Epoch: 072, Train Loss: 0.7036
Epoch: 073, Train Loss: 0.7032
Epoch: 074, Train Loss: 0.7028
Epoch: 075, Train Loss: 0.7022


...

Val Mean Rank: 2053.02, Val Mean Reciprocal Rank: 0.09, Val Hits@10: 0.1706
Epoch: 476, Train Loss: 0.6228
Epoch: 477, Train Loss: 0.6229
Epoch: 478, Train Loss: 0.6225
Epoch: 479, Train Loss: 0.6223
Epoch: 480, Train Loss: 0.6223
Epoch: 481, Train Loss: 0.6223
Epoch: 482, Train Loss: 0.6221
Epoch: 483, Train Loss: 0.6222
Epoch: 484, Train Loss: 0.6219
Epoch: 485, Train Loss: 0.6218
Epoch: 486, Train Loss: 0.6217
Epoch: 487, Train Loss: 0.6216
Epoch: 488, Train Loss: 0.6215
Epoch: 489, Train Loss: 0.6213
Epoch: 490, Train Loss: 0.6214
Epoch: 491, Train Loss: 0.6211
Epoch: 492, Train Loss: 0.6211
Epoch: 493, Train Loss: 0.6210
Epoch: 494, Train Loss: 0.6209
Epoch: 495, Train Loss: 0.6209
Epoch: 496, Train Loss: 0.6205
Epoch: 497, Train Loss: 0.6207
Epoch: 498, Train Loss: 0.6206
Epoch: 499, Train Loss: 0.6202
Epoch: 500, Train Loss: 0.6202


100%|██████████| 17535/17535 [04:06<00:00, 71.11it/s]


Val Mean Rank: 2024.23, Val Mean Reciprocal Rank: 0.09, Val Hits@10: 0.1752


100%|██████████| 20466/20466 [04:35<00:00, 74.42it/s]

Test Mean Rank: 2069.01, Test Mean Reciprocal Rank: 0.09, Test Hits@10: 0.1738


In [30]:
path = osp.join(os.getcwd(), f'{dataset_name}_embedding_model_weights.pth')
torch.save(model.state_dict(), path)

# Fetch a batch from your train_loader
for batch in train_loader:
    # Assuming batch contains head_index, rel_type, tail_index, and possibly other data
    head_index, rel_type, tail_index = batch
    break  # Only need one batch for this purpose

# Use the fetched batch to provide dummy inputs for the export
# Ensure these variables are moved to the same device as your model if necessary
torch.onnx.export(
    model,
    (head_index, rel_type, tail_index),  # Use actual data as dummy inputs
    f'{dataset_name}_embedding_model_weights.onnx',
    opset_version=11,
    do_constant_folding=True,
    input_names=["head_index", "rel_type", "tail_index"],  # Adjust input names as needed
    dynamic_axes={
        "head_index": {0: "batch_size"}, 
        "rel_type": {0: "batch_size"}, 
        "tail_index": {0: "batch_size"}
    }
)
wandb.save(f'{dataset_name}_embedding_model_weights.onnx')
wandb.finish()

test_hits_at_10,▁
test_loss,▁
test_mean_rank,▁
test_mrr,▁
train_epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▇▆▆▆▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
val_hits_at_10,▁▁▁▁▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇██████
val_loss,██▆▆▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
val_mean_rank,██▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mrr,▁▁▁▁▂▂▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇██████
test_hits_at_10,0.17385
